<a href="https://colab.research.google.com/github/garfield-gray/Optimization/blob/main/Convex/IntProg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
# suit your code in order to handele the negatives!!

# IntProg

In [118]:
from scipy.optimize import LinearConstraint, milp
import numpy as np

In [119]:
c = np.array([3, 4])
A = np.array([[-3, -1], [-1, -2]])
b_u = np.array([-4, -4])
b_l = np.full_like(b_u, -np.inf, dtype=float)
# b_l = np.full_like(b_u, 0, dtype=float)

In [120]:
constraints = LinearConstraint(A, b_l, b_u)

In [121]:
integrality = np.ones_like(c)
res = milp(c=c, constraints=constraints, integrality=integrality)
res.x

array([2., 1.])

In [122]:
res = milp(c=c, constraints=constraints)  # OR:
# from scipy.optimize import linprog; res = linprog(c, A, b_u)
res.x

array([0.8, 1.6])

# Linprog

In [123]:
from scipy.optimize import linprog
c = [3, 4]
A = [[-3, -1], [-1, -2]]
b = [-4, -4]
x0_bounds = (0, None)
x1_bounds = (0, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=[x0_bounds, x1_bounds])
res.fun


8.8

In [124]:
res.x

array([0.8, 1.6])

In [125]:
res.message

'Optimization terminated successfully. (HiGHS Status 7: Optimal)'

# The Algorithm

In [126]:
def CoreSimplex(tableau):
    tableau = np.array(tableau)
    m, n = tableau.shape
    m -=1
    n -=1
    while True:
        # Check if we have an optimal solution (all entries in the objective row are negative)

        if np.all(tableau[0, :-1] <= 0):
            break

        # Pivot column (most negative entry in the objective row)
        pivot_col = np.argmax(tableau[0, :-1])



        # Pivot row
        ratios = np.divide(tableau[1:, -1], tableau[1:, pivot_col])

        valid_ratios = ratios[tableau[1:, pivot_col] > 0]

        # Check if there's no valid ratio which means unbounded
        if len(valid_ratios) == 0:
            raise ValueError("The problem is unbounded.")


        ########could be written better################
        pivot_row = np.where(ratios == valid_ratios.min())[0][0] +1

        # Pivot element
        pivot_element = tableau[pivot_row, pivot_col]

        # Update the pivot row
        tableau[pivot_row] /= pivot_element
        # Update the other rows
        for i in range(m + 1):
            if i != pivot_row:
                tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

    return tableau


In [127]:
t = np.array(
    [[4.0,  0, -2, -1,  0,  0,  0,  4],
     [ 1,  0, -1,  0,  1,  0,  0,  1],
     [ 2,  1,  1,  0,  0,  1,  0,  2],
     [ 1, -1, -2, -1,  0,  0,  1,  1]])
print(np.round(CoreSimplex(t), 4))

[[ 0.     -0.6667  0.     -1.     -2.6667 -0.6667  0.      0.    ]
 [ 1.      0.3333  0.      0.      0.3333  0.3333  0.      1.    ]
 [ 0.      0.3333  1.      0.     -0.6667  0.3333  0.      0.    ]
 [ 0.     -0.6667  0.     -1.     -1.6667  0.3333  1.      0.    ]]


In [158]:

# needs to checked if variables are all positive
def simplex(c, A, b):
    # Number of variables
    n = len(c)
    # Number of constraints
    m = len(b)
    # creation of bfs(Basic feasible solution through Artificial variables)
    Rtableau = np.zeros((m + 1, n + m + 1))
    Rtableau[1:, :n] = A
    Rtableau[1:, -1] = b
    p = np.eye(m+1)
    p[np.where(b<0)+np.ones_like(np.where(b<0))] *= -1
    Rtableau = p @ Rtableau
    Rtableau[1:, n:n+m] = np.eye(m)
    #
    Rtableau[0, n:n+m] = -np.ones(m)
    for i in range(m):
        Rtableau[0] += Rtableau[i+1]


    Rtableau = CoreSimplex(Rtableau)
    # checking the feasibility
    # if
    tableau = np.delete(Rtableau, np.s_[n:n+m], axis=1)


    tableau[0, :-1] = -c
    for i in range(m):
        # print(i)
        # print((tableau[0, np.where(tableau[i+1]==1)[0]]))
        tableau[0] -= tableau[0, np.where(tableau[i+1]==1)[0]] * tableau[i+1]
    # print(tableau)
    tableau = CoreSimplex(tableau)


    return tableau


In [159]:
A = np.array([
    [3, 1,-1, 0],
    [1, 2, 0,-1],

])
b = np.array([4, 4])
c = np.array([3, 4, 0, 0])
np.round(simplex(c, A, b),4)

array([[ 0. ,  0. , -0.4, -1.8,  8.8],
       [ 1. ,  0. , -0.4,  0.2,  0.8],
       [ 0. ,  1. ,  0.2, -0.6,  1.6]])

In [175]:
# needs to checked if variables are all positive
def intSimplex(c, A, b):
    # Number of variables
    n = len(c)
    # Number of constraints
    m = len(b)
    # creation of bfs(Basic feasible solution through Artificial variables)
    Rtableau = np.zeros((m + 1, n + m + 1))
    Rtableau[1:, :n] = A
    Rtableau[1:, -1] = b
    p = np.eye(m+1)
    p[np.where(b<0)+np.ones_like(np.where(b<0))] *= -1
    Rtableau = p @ Rtableau
    Rtableau[1:, n:n+m] = np.eye(m)
    #
    Rtableau[0, n:n+m] = -np.ones(m)
    for i in range(m):
        Rtableau[0] += Rtableau[i+1]


    Rtableau = CoreSimplex(Rtableau)
    # checking the feasibility
    # if
    tableau = np.delete(Rtableau, np.s_[n:n+m], axis=1)


    tableau[0, :-1] = -c
    for i in range(m):
        # print(i)
        # print((tableau[0, np.where(tableau[i+1]==1)[0]]))
        tableau[0] -= tableau[0, np.where(tableau[i+1]==1)[0]] * tableau[i+1]
    # print(tableau)
    tableau = CoreSimplex(tableau)

    while True:
        # Check if we have an optimal solution (all entries in the objective row are negative)
        if np.all(np.modf(tableau[1:, -1])[0] == 0):
            break
        ########could be written better################
        fracIdx = (np.where(np.modf(tableau[1:, -1])[0] != 0)[0]+1)[0]
        print(tableau[fracIdx]-np.floor(tableau[fracIdx]))
        break
    return tableau


In [176]:
A = np.array([
    [3, 1,-1, 0],
    [1, 2, 0,-1],

])
b = np.array([4, 4])
c = np.array([3, 4, 0, 0])
np.round(intSimplex(c, A, b),4)

[0.  0.  0.6 0.2 0.8]


array([[ 0. ,  0. , -0.4, -1.8,  8.8],
       [ 1. ,  0. , -0.4,  0.2,  0.8],
       [ 0. ,  1. ,  0.2, -0.6,  1.6]])

In [ ]:
import numpy as np

# Example array
arr = np.array([-2.5, 2.5, 3.0, 4.2, -3.7])

# Get the integral part using floor
integral_part = np.floor(arr)

# Get the fractional part by subtracting the integral part from the original array
fractional_part = arr - integral_part

print("Array:", arr)
print("Integral part:", integral_part)
print("Fractional part:", fractional_part)


Array: [-2.5  2.5  3.   4.2 -3.7]
Integral part: [-3.  2.  3.  4. -4.]
Fractional part: [0.5 0.5 0.  0.2 0.3]
